In [ ]:
from google.colab import files
uploaded = files.upload()


Saving cleaned_embeddings.csv to cleaned_embeddings.csv


In [ ]:
#1:Read data in
import pandas as pd
import ast
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.datasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow.keras.optimizers
from tensorflow.keras.callbacks import EarlyStopping
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np


embeddings = pd.read_csv("cleaned_embeddings.csv")
# embeddings['Embedding'] = embeddings['Embedding'].str().replace('[', '').replace(']', '').split(',').astype(float64)
embeddings['Embedding'] = embeddings['Embedding'].apply(ast.literal_eval)

# Convert to numpy array
embedding_matrix = np.vstack(embeddings['Embedding'].values)

In [ ]:
#2: First unmodified model
from sklearn.preprocessing import StandardScaler

X = np.array(embedding_matrix).astype('float32')
y = np.array(embeddings['SPY'].to_numpy()).astype('float32').reshape(-1, 1)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Normalize inputs
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Define model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='linear'))  #Linear activation function

# Compile and train
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=['mae'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)

model.fit(
    X_train_scaled, y_train,
    batch_size=64,
    epochs=150,
    verbose=1,
    validation_data=(X_val_scaled, y_val),
    callbacks=[early_stopping]
)

# Evaluate on validation set
val_loss, val_mae = model.evaluate(X_val_scaled, y_val)

# Predictions on test set
y_pred = model.predict(X_test_scaled)

# Save predictions alongside actuals
results = pd.DataFrame({
    'predicted_change': y_pred.flatten(),
    'actual_change': y_test.flatten()
})

print(results.head())


results = pd.DataFrame(y_test)
results['predictions'] = model.predict(X_test_scaled)
results

Epoch 1/150


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


33/33 ━━━━━━━━━━━━━━━━━━━━ 6s 48ms/step - loss: 1.9568 - mae: 0.9955 - val_loss: 1.3417 - val_mae: 0.7596
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 1.0777 - mae: 0.7138 - val_loss: 1.3281 - val_mae: 0.7464
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 1.0178 - mae: 0.6858 - val_loss: 1.3382 - val_mae: 0.7463
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 1.1330 - mae: 0.7161 - val_loss: 1.3693 - val_mae: 0.7524
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.0962 - mae: 0.7027 - val_loss: 1.4531 - val_mae: 0.7886
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.0900 - mae: 0.6923 - val_loss: 1.3668 - val_mae: 0.7414
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.1319 - mae: 0.6977 - val_loss: 1.2896 - val_mae: 0.7411
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9335 - mae: 0.6673 - val_loss: 1.3233 - val_mae: 0.7461
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9716 - mae: 

,0,predictions
0,0.864567,0.050980
1,-1.110721,0.050980
2,-0.238876,0.051013
3,1.317129,0.000750
4,-0.059061,-0.056036
...,...,...
257,-0.284404,0.174802
258,0.559663,0.050980
259,0.213990,-0.243255
260,0.040169,0.058686


In [ ]:
results = pd.DataFrame({
    'actual_change': y_test.flatten(),
    'predicted_change': model.predict(X_test_scaled).flatten()
})

from sklearn.metrics import r2_score
r2 = r2_score(results['actual_change'], results['predicted_change'])
print("R² Score:", r2)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
R² Score: -0.03274738788604736


In [ ]:
#2: Second model - lagged
from sklearn.preprocessing import StandardScaler

X = np.array(embedding_matrix).astype('float32')
y = np.array(embeddings['SPY'].to_numpy()).astype('float32').reshape(-1, 1)

#Here, I lagged the stock data by -1. So on 11/3, you see the percent change from 11/1 to 11/2.
X = X[:-1]
y = y[1:]

# Train/val/test split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Normalize inputs
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Define model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='linear'))  #Linear activation function

# Compile and train
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=['mae'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)

model.fit(
    X_train_scaled, y_train,
    batch_size=64,
    epochs=150,
    verbose=1,
    validation_data=(X_val_scaled, y_val),
    callbacks=[early_stopping]
)

# Evaluate on validation set
val_loss, val_mae = model.evaluate(X_val_scaled, y_val)

# Predictions on test set
y_pred = model.predict(X_test_scaled)

# Save predictions alongside actuals
results = pd.DataFrame({
    'predicted_change': y_pred.flatten(),
    'actual_change': y_test.flatten()
})

print(results.head())


results = pd.DataFrame(y_test)
results['predictions'] = model.predict(X_test_scaled)
results

Epoch 1/150


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 2.6300 - mae: 1.0902 - val_loss: 1.5483 - val_mae: 0.7833
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.1708 - mae: 0.7134 - val_loss: 1.5357 - val_mae: 0.7856
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.1358 - mae: 0.6843 - val_loss: 1.5818 - val_mae: 0.8086
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9349 - mae: 0.6653 - val_loss: 1.5579 - val_mae: 0.7868
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.9297 - mae: 0.6671 - val_loss: 1.5509 - val_mae: 0.7740
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.9545 - mae: 0.6824 - val_loss: 1.5995 - val_mae: 0.8067
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9229 - mae: 0.6519 - val_loss: 1.4947 - val_mae: 0.7838
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.0377 - mae: 0.6912 - val_loss: 1.8370 - val_mae: 0.8942
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.2454 - mae: 0

,0,predictions
0,-0.182374,0.192820
1,1.531631,0.055541
2,-1.075168,0.055541
3,0.505230,0.055332
4,0.115729,0.076169
...,...,...
257,0.004600,0.055541
258,0.454429,0.034377
259,0.234536,0.055541
260,-0.853242,0.133302


In [ ]:
results = pd.DataFrame({
    'actual_change': y_test.flatten(),
    'predicted_change': model.predict(X_test_scaled).flatten()
})

from sklearn.metrics import r2_score
r2 = r2_score(results['actual_change'], results['predicted_change'])
print("R² Score:", r2)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
R² Score: -0.027127623558044434


In [ ]:
#3: third model, lagged, tanh
from sklearn.preprocessing import StandardScaler

X = np.array(embedding_matrix).astype('float32')
y = np.array(embeddings['SPY'].to_numpy()).astype('float32').reshape(-1, 1)

#Here, I lagged the stock data by -1. So on 11/3, you see the percent change from 11/1 to 11/2.
X = X[:-1]
y = y[1:]

# Train/val/test split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Normalize inputs
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Define model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='tanh'))  #tanh activation function

# Compile and train
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=['mae'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)

model.fit(
    X_train_scaled, y_train,
    batch_size=64,
    epochs=150,
    verbose=1,
    validation_data=(X_val_scaled, y_val),
    callbacks=[early_stopping]
)

# Evaluate on validation set
val_loss, val_mae = model.evaluate(X_val_scaled, y_val)

# Predictions on test set
y_pred = model.predict(X_test_scaled)

# Save predictions alongside actuals
results = pd.DataFrame({
    'predicted_change': y_pred.flatten(),
    'actual_change': y_test.flatten()
})

print(results.head())


results = pd.DataFrame(y_test)
results['predictions'] = model.predict(X_test_scaled)
results

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - loss: 1.8253 - mae: 0.9929 - val_loss: 1.9883 - val_mae: 1.0177
Epoch 2/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1.6261 - mae: 0.9741 - val_loss: 1.7067 - val_mae: 0.8541
Epoch 3/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1.4350 - mae: 0.8251 - val_loss: 1.7208 - val_mae: 0.8750
Epoch 4/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 1.3014 - mae: 0.7446 - val_loss: 1.6160 - val_mae: 0.8453
Epoch 5/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.0303 - mae: 0.6823 - val_loss: 1.6812 - val_mae: 0.8552
Epoch 6/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.0653 - mae: 0.6950 - val_loss: 1.7369 - val_mae: 0.8768
Epoch 7/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.8945 - mae: 0.6298 - val_loss: 1.6920 - val_mae: 0.8571
Epoch 8/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.7452 - mae: 0.5673 - val_loss: 1.8657 - val_mae: 0.9135
Epoch 9/150
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0

,0,predictions
0,-0.182374,0.193667
1,1.531631,0.262159
2,-1.075168,0.190116
3,0.505230,0.328109
4,0.115729,0.441323
...,...,...
257,0.004600,0.119623
258,0.454429,-0.727037
259,0.234536,0.332709
260,-0.853242,0.056609


In [ ]:
results = pd.DataFrame({
    'actual_change': y_test.flatten(),
    'predicted_change': model.predict(X_test_scaled).flatten()
})

from sklearn.metrics import r2_score
r2 = r2_score(results['actual_change'], results['predicted_change'])
print("R² Score:", r2)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
R² Score: -0.12399816513061523
